In [1]:
%pip install -U langchain-community langgraph langchain-openai tavily-python

  Using cached langchain_community-0.2.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached langgraph-0.1.8-py3-none-any.whl.metadata (13 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.2.7-py3-none-any.whl (2.2 MB)
Using cached langgraph-0.1.8-py3-none-any.whl (91 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [5]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

/Users/rickchakra/Projects/aiug_workshop/venv_new/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721306754, 'localtime': '2024-07-18 5:45'}, 'current': {'last_updated_epoch': 1721306700, 'last_updated': '2024-07-18 05:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1016.0, 'pressure_in': 29.99, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 75, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'windchill_c': 14.1, 'windchill_f': 57.4, 'heatindex_c': 13.9, 'heatindex_f': 57.1, 'dewpoint_c': 12.5, 'dewpoint_f': 54.4, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 4.6, 'gust_kph': 7.4}}"}, {'url': 'https://www.accuweather.com/en/us/san-fra

In [7]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [11]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [12]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [13]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [14]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [15]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in new york city?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New York City'}, 'id': 'call_aQA4oYMmdFD1jIu1OWBCSCs6', 'type': 'tool_call'}]


In [16]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools)

In [17]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='d63983be-f077-414a-9e8b-5efb435c6855'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_18cc0f1fa0', 'finish_reason': 'stop', 'logprobs': None}, id='run-af260025-7001-48de-94e9-c3aadcba021e-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [20]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="I want to go running in Seattle but i don't know if it's raining?")]}
)
response["messages"]

[HumanMessage(content="I want to go running in Seattle but i don't know if it's raining?", id='131e4a04-a237-4dca-ae39-be0b551559af'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EZkW6N9BhRSNM6CfXys9qlQ3', 'function': {'arguments': '{"query":"current weather in Seattle"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 94, 'total_tokens': 115}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5e997b69d8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b13dfc1b-b4bf-452d-b8a8-c149d296bb6a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seattle'}, 'id': 'call_EZkW6N9BhRSNM6CfXys9qlQ3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 21, 'total_tokens': 115}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Seattle\', \'regio